# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `~ mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'epa', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [7]:
# review a document in the establishments collection
db.establishments.find_one()

{'_id': ObjectId('657043fcf2803e50af27d06c'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': 5,
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 1.195625, 'latitude': 51.083812},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_halal = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [10]:
# Insert the new restaurant into the collection
new_entry = establishments.insert_one(new_halal)

In [11]:
# Check that the new restaurant was inserted
print('Inserted Document ID:', new_entry.inserted_id)

Inserted Document ID: 6577ed986f240b030aa8a326


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': {'$regex': 'Restaurant/Cafe/Canteen'}}
fields = {'BusinessTypeID': 1, 'BusinessType': 1}
limit = 3

results = list(establishments.find(query, fields).limit(limit))
pprint(results)

[{'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('657043fcf2803e50af27d06c')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('657043fcf2803e50af27d073')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('657043fcf2803e50af27d082')}]


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID

query = {'_id': new_entry.inserted_id}
update = {'$set': {'BusinessTypeID': 1 }}
result = establishments.update_one(query, update)
print('Matched', result.matched_count, 'documents and modified', result.modified_count, 'documents')

Matched 1 documents and modified 1 documents


In [14]:
# Confirm that the new restaurant was updated
pprint(list(establishments.find(query)))

[{'AddressLine1': 'Penang Flavours',
  'AddressLine2': '146A Plumstead Rd',
  'AddressLine3': 'London',
  'AddressLine4': '',
  'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  'Distance': 4623.972328074718,
  'LocalAuthorityCode': '511',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'NewRatingPending': True,
  'Phone': '',
  'PostCode': 'SE18 7DY',
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('6577ed986f240b030aa8a326'),
  'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
  'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}]


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
target_value = 'Dover'
query = {'LocalAuthorityName': target_value}
count = establishments.count_documents(query)
print(f'Number of documents with city "{target_value}": {count}')

Number of documents with city "Dover": 0


In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_dover = establishments.delete_many(query)
print(f'Number of documents deleted with city "{target_value}": {delete_dover.deleted_count}')

Number of documents deleted with city "Dover": 0


In [17]:
# Check if any remaining documents include Dover
query = {'LocalAuthorityName': target_value}
count = establishments.count_documents(query)
print(f'Number of documents with city "{target_value}": {count}')

Number of documents with city "Dover": 0


In [18]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('657043fcf2803e50af27d06c'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None,


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Change the data type from String to Decimal for longitude and latitude

# update latitude
establishments.update_many({}, [ {'$set': {'geocode.latitude' : {'$toDouble': '$geocode.latitude'}}}])
print('number of fields where latitude was changed from str to Double: ', result.modified_count)



number of docs where latitude was changed from str to Double:  1


In [20]:
# update longitude
establishments.update_many({}, [ {'$set': {'geocode.longitude' : {'$toDouble': '$geocode.longitude'}}}])
print('number of fields longitude was changed from str to int: ', result.modified_count)

number of docs where RatingValue was changed from str to int:  1


Use `update_many` to convert `RatingValue` to integer numbers.

In [21]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [22]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [ {'$set': {'RatingValue' : {'$toInt': '$RatingValue'}}}])
print('number of fields changed from str to int: ', result.modified_count)

number of fields changed from str to int:  1


In [23]:
# Check that the coordinates and rating value are now numbers (they are, since the '' have been removed and formatting matches in the MongoDB Compass app)
query = {}
fields = {'RatingValue': 1, 'geocode.latitude': 1, 'geocode.longitude': 1}
results = establishments.find(query, fields)
limit = 5

pprint(list(establishments.find(query, fields).limit(limit)))

[{'RatingValue': 5,
  '_id': ObjectId('657043fcf2803e50af27d06c'),
  'geocode': {'latitude': 51.083812, 'longitude': 1.195625}},
 {'RatingValue': 5,
  '_id': ObjectId('657043fcf2803e50af27d06d'),
  'geocode': {'latitude': 51.085797, 'longitude': 1.194762}},
 {'RatingValue': 4,
  '_id': ObjectId('657043fcf2803e50af27d06f'),
  'geocode': {'latitude': 51.08084, 'longitude': 1.188537}},
 {'RatingValue': 5,
  '_id': ObjectId('657043fcf2803e50af27d070'),
  'geocode': {'latitude': 51.08084, 'longitude': 1.188537}},
 {'RatingValue': 5,
  '_id': ObjectId('657043fcf2803e50af27d071'),
  'geocode': {'latitude': 51.0783519967076, 'longitude': 1.18590330311705}}]
